In [ ]:
import io
import os
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display
import ipywidgets as widgets
from os import listdir
from os.path import isfile, join

from config import config

%run helpers/parsing.ipynb

# Define global variables for later use
dataset_config = None
filename: str = None
buffer: bytes = None
xl: pd.ExcelFile = None
sheet: pd.DataFrame = None
df: pd.DataFrame = None
column_name: str = None
sheet_name: str = None
gaps_indices = []

sheet_select: widgets.Dropdown = None
column_select: widgets.Dropdown = None

# Configuration form
config_select = widgets.Dropdown(
    options=list(config["datasets_configs"].keys()),
    value=list(config["datasets_configs"].keys())[0],
    description='Config:'
)
display(config_select)
dataset_config = config["datasets_configs"][config_select.value]

def on_config_select_change(evt):
    global dataset_config
    dataset_config = config["datasets_configs"][evt.new]
config_select.observe(on_config_select_change, 'value')


# Create upload status text
upload_status = widgets.HTML(value='')


# Create the file explorer tab
file_select = widgets.Dropdown(
    options=[f for f in listdir(config["upload_dir"]) if isfile(join(config["upload_dir"], f))],
    value=None
)

def on_file_select_change(evt):
    global file_select, filename, buffer
    file_select.disabled = True
    filename = join(config["upload_dir"], evt.new)
    upload_status.value = 'Upload starting...'
    with open(filename, 'rb') as file:
        buffer = file.read()
        upload_status.value = 'Upload successful'
    parsing_status.value = 'Reading file data...'
    parse_uploaded_file_interactive(filename, buffer)
    file_select.disabled = False
file_select.observe(on_file_select_change, 'value')

file_explorer_form = widgets.VBox(
    children=[
        widgets.HTML(value='Uploaded file selector:'),
        widgets.HBox(children=[
            file_select
        ])
    ]
)


# Create the upload tab
upload_button = widgets.FileUpload(accept='.csv,.xlsx', multiple=False)
upload_form = widgets.VBox(
    children=[
        widgets.HTML(value='Upload file (limit 10Mo):'),
        upload_button,
        upload_status
    ]
)

def on_upload_change(evt):
    global upload_button, filename, buffer
    upload_button.disabled = True
    filename = list(evt.owner.value.keys())[0]
    buffer = evt.owner.data[0]
    with open(join(config["upload_dir"], filename), 'wb') as f:
        f.write(buffer)
    upload_status.value = 'Upload successful'
    parsing_status.value = 'Reading file data...'
    parse_uploaded_file_interactive(filename, buffer)
    upload_button.disabled = False
upload_button.observe(on_upload_change, names='_counter')


# Create the file path tab
default_file_path = '' # tweak this
filepath_text_input = widgets.Text(value=default_file_path, continuous_update=False)
filepath_submit_button = widgets.Button(description='Submit')
filepath_form = widgets.VBox(
    children=[
        widgets.HTML(value='Relative file path:'),
        widgets.HBox(children=[
            filepath_text_input,
            filepath_submit_button
        ]),
        upload_status
    ]
)

def on_filepath_submit(new_value: str):
    global filepath_submit_button, filename, buffer
    filepath_submit_button.disabled = True
    filename = new_value
    upload_status.value = 'Upload starting...'
    with open(filename, 'rb') as file:
        buffer = file.read()
        upload_status.value = 'Upload successful'
    parsing_status.value = 'Reading file data...'
    parse_uploaded_file_interactive(filename, buffer)
    filepath_submit_button.disabled = False
filepath_submit_button.on_click(lambda _: on_filepath_submit(filepath_text_input.value))


# Create tabs container
tabs_titles = ['Explorer', 'Upload', 'File path']
tabs_container = widgets.Tab(children=[file_explorer_form, upload_form, filepath_form])
for i in range(len(tabs_titles)):
    tabs_container.set_title(i, tabs_titles[i])
display(tabs_container)


# Create parsing status text
parsing_status = widgets.HTML(value='')
display(parsing_status)
